## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3)))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))
#flatten
classifier.add(Flatten())

#FC
#classifier.add(Dense('自己設計FC層參數')) #output_dim=100,activation=relu
classifier.add(Dense(units=100, activation='relu')) 
#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 102s 2ms/step - loss: 1.4829 - acc: 0.4757
Epoch 2/100
50000/50000 [==============================] - 105s 2ms/step - loss: 1.0474 - acc: 0.6296
Epoch 3/100
50000/50000 [==============================] - 101s 2ms/step - loss: 0.9110 - acc: 0.6763
Epoch 4/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.8220 - acc: 0.7078
Epoch 5/100
50000/50000 [==============================] - 101s 2ms/step - loss: 0.7526 - acc: 0.7359
Epoch 6/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.6729 - acc: 0.7652
Epoch 7/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.6068 - acc: 0.7887
Epoch 8/100
50000/50000 [==============================] - 102s 2ms/step - loss: 0.5250 - acc: 0.8162
Epoch 9/100
50000/50000 [==============================] - 107s 2ms/step - loss: 0.4605 - 

50000/50000 [==============================] - 108s 2ms/step - loss: 0.0228 - acc: 0.9926
Epoch 78/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0253 - acc: 0.9924
Epoch 79/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.0279 - acc: 0.9909
Epoch 80/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.0224 - acc: 0.9937
Epoch 81/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0225 - acc: 0.9929
Epoch 82/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.0262 - acc: 0.9923
Epoch 83/100
50000/50000 [==============================] - 102s 2ms/step - loss: 0.0233 - acc: 0.9928
Epoch 84/100
50000/50000 [==============================] - 105s 2ms/step - loss: 0.0195 - acc: 0.9940
Epoch 85/100
50000/50000 [==============================] - 109s 2ms/step - loss: 0.0231 - acc: 0.9930
Epoch 86/100
50000/50000 [==============================] - 107s 2ms/step - loss: 0.02

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.5188237e-11, 3.3006562e-32, 4.2907053e-23, 1.0000000e+00,
        2.6009280e-14, 0.0000000e+00, 3.6963075e-16, 5.7652580e-38,
        2.8377478e-15, 6.1796451e-18]], dtype=float32)